In [82]:

import sys

sys.path.append("D:/StateOfStates/")

import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score

from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian as Partisian
from sklearn.linear_model import LinearRegression
import addfips


In [83]:
states = { 'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa', 'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi', 'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming'
}


states = {v: k for k, v in states.items()}
states


{'Alaska': 'AK',
 'Alabama': 'AL',
 'Arkansas': 'AR',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'District of Columbia': 'DC',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Iowa': 'IA',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Massachusetts': 'MA',
 'Maryland': 'MD',
 'Maine': 'ME',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Missouri': 'MO',
 'Northern Mariana Islands': 'MP',
 'Mississippi': 'MS',
 'Montana': 'MT',
 'National': 'NA',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Nebraska': 'NE',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'Nevada': 'NV',
 'New York': 'NY',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 

In [84]:
def translate_ICD10(x, conversion):
    try:
        return conversion[x]
    except:
        return x
    
    



def clean_state_data(df):
    af = addfips.AddFIPS()
    try:
        df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])
    except: 
        df = df.dropna(subset=["State", "Population"])
    #df = df[df["Crude Rate"] != "Unreliable"]
    df.Population = df.Population.astype(float)
    df["Rate"] = (df["Deaths"]/df["Population"])*100000
    df["Rate"] = df["Rate"].round(2)
    df["fips"] = df.apply(lambda x: af.get_state_fips(x["State"]), axis=1)
    df["Year"] = 2019
    df["State_Abv"] = df.apply(lambda x: states[x["State"]], axis=1)
    df = df.sort_values(by=["Rate"], ascending=False)
    try:
        df.drop(columns=["Notes", "State Code", ], inplace=True)
    except:
        None
   
    try:
        f = open("data/Life/Deaths/ICD10Translation.json")
        translation = json.load(f)
        df["ICD-10 Common"] = df["ICD-10 113 Cause List"].apply(lambda x: translate_ICD10(x, translation))
        df.rename(columns={"ICD-10 Common":"Cause Of Death"}, inplace=True)
    except:
        None
    return df
    


def top_n(df, n):
    #Filter out low occuring chapters
    df = df.groupby("ICD-10 113 Cause List Code").filter(lambda x: len(x)>1)
    df = df.sort_values(by = ["Rate"], ascending = False)
    df = df.head(n).copy()
    top_codes = list(df["ICD-10 113 Cause List Code"].unique())
    df = df[df["ICD-10 113 Cause List Code"].isin(top_codes)]
    return df


firearms = pd.read_csv(r"../data/Life/Deaths/FireArmState.txt", delimiter="	", na_values = ['Not Applicable'])
df = pd.read_csv(r"../data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = pd.read_csv(r"../data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = clean_state_data(state_pop)
yearly_pop = pd.read_csv(r"../data/Life/Deaths/2019State1YearPop.txt", delimiter="	", na_values = ['Not Applicable'])
state_deaths = pd.read_csv(r"../data/Life/Deaths/2019StateDeaths.txt", delimiter="	", na_values = ['Not Applicable'])


df = clean_state_data(df)
firearms = clean_state_data(firearms)
state_pop = clean_state_data(state_pop)
yearly_pop = clean_state_data(yearly_pop)
state_deaths = clean_state_data(state_deaths)
state_pop["Total Pop"] = state_pop["Population"]

#top_df = top_n(df, 5)
#county = df.groupby(by=["2006 Urbanization Code", "ICD Sub-Chapter"]).mean().reset_index().sort_values(by=["Rate"], ascending=False)
state_pop.sample(5)
firearms = firearms.merge(state_pop[["State", "Total Pop"]], left_on="State",right_on="State")

In [85]:
state_pop

,State,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv,Total Pop
48,West Virginia,23404.0,1792147.0,1305.9,1305.92,54,2019,WV,1792147.0
19,Maine,15065.0,1344212.0,1120.7,1120.73,23,2019,ME,1344212.0
24,Mississippi,32964.0,2976149.0,1107.6,1107.61,28,2019,MS,2976149.0
0,Alabama,54108.0,4903185.0,1103.5,1103.53,01,2019,AL,4903185.0
17,Kentucky,48990.0,4467673.0,1096.5,1096.54,21,2019,KY,4467673.0
3,Arkansas,32888.0,3017804.0,1089.8,1089.80,05,2019,AR,3017804.0
35,Ohio,123717.0,11689100.0,1058.4,1058.40,39,2019,OH,11689100.0
42,Tennessee,71935.0,6829174.0,1053.3,1053.35,47,2019,TN,6829174.0
38,Pennsylvania,133983.0,12801989.0,1046.6,1046.58,42,2019,PA,12801989.0
36,Oklahoma,40930.0,3956971.0,1034.4,1034.38,40,2019,OK,3956971.0


In [86]:
part = Partisian.get_state_part_score()
final = pd.merge(part, firearms, on="fips")
final[final["State_x"] == "VT"]
state_totals = final.groupby(by=["fips","State_y"]).mean().reset_index().sort_values(by="Rate")
state_totals["Rate"] = (state_totals["Deaths"]/state_totals["Total Pop"])*100000

C:\Users\zeusg\AppData\Local\Temp/ipykernel_38384/127021580.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Part_Score"], y=state_totals["Rate"], mode = "markers",  hovertext=state_totals["State_y"], marker=dict(size=state_totals["Total Pop"]/1000000, colorscale='bluered', color=state_totals["Part_Score"])))

In [7]:
state_totals.sort_values("Rate", ascending=False, inplace=True)
fig = go.Figure()
fig.add_trace(go.Bar(x=state_totals["State_y"], y=state_totals["Rate"], name="Rate"))
fig.show()

In [8]:
state_totals[state_totals["State_y"] == "Vermont"]

,fips,State_y,Part_Score,Year_x,Deaths,Population,Rate,Year_y,Total Pop
43,50,Vermont,-0.4,2024.0,137.0,6904365.5,21.955515,2019.0,623989.0


In [9]:
state_totals

,fips,State_y,Part_Score,Year_x,Deaths,Population,Rate,Year_y,Total Pop
43,50,Vermont,-0.400000,2024.0,137.000000,6.904366e+06,21.955515,2019.0,623989.0
32,38,North Dakota,1.000000,2024.0,87.750000,3.945920e+06,11.514811,2019.0,762062.0
48,56,Wyoming,0.866667,2024.0,50.300000,1.349089e+06,8.691010,2019.0,578759.0
17,22,Louisiana,0.333333,2024.0,307.130952,2.346557e+06,6.606680,2019.0,4648794.0
23,28,Mississippi,0.800000,2024.0,181.063291,1.558334e+06,6.083811,2019.0,2976149.0
7,10,Delaware,-1.000000,2024.0,56.611111,7.961586e+05,5.813638,2019.0,973764.0
1,02,Alaska,0.466667,2024.0,41.755556,5.553994e+05,5.707859,2019.0,731545.0
0,01,Alabama,0.866667,2024.0,251.875000,2.504941e+06,5.136967,2019.0,4903185.0
29,35,New Mexico,-0.466667,2024.0,106.769231,1.217324e+06,5.091938,2019.0,2096829.0
19,24,Maryland,-0.533333,2024.0,295.526316,3.157371e+06,4.888223,2019.0,6045680.0


In [10]:
12/133915.0

8.960908038681252e-05

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=state_totals["Population"], y=state_totals["Rate"], mode = "markers", hovertext=state_totals["State_y"], marker=dict(colorscale='bluered', color=state_totals["Population"])))

In [12]:
usa_total = yearly_pop.groupby("Single-Year Ages").sum().reset_index()
usa_total["age"] = usa_total["Single-Year Ages"].str.replace("< 1 year","0").str.split(" ").str[0].astype(int)
usa_total.sort_values("age", inplace=True)
usa_total["Rate"] = (usa_total["Deaths"]/usa_total["Population"])*100000
usa_total

C:\Users\zeusg\AppData\Local\Temp/ipykernel_38384/196067746.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Single-Year Ages,Deaths,Population,Rate,Year,age
84,< 1 year,20921.0,3783052.0,553.019097,102969,0
0,1 year,1401.0,3621711.0,38.683374,74703,1
11,2 years,835.0,3463061.0,24.111617,62589,2
22,3 years,596.0,3228814.0,18.458790,48456,3
33,4 years,488.0,3201427.0,15.243202,48456,4
...,...,...,...,...,...,...
78,80 years,68349.0,1475278.0,4632.957314,102969,80
79,81 years,70621.0,1381641.0,5111.385664,102969,81
80,82 years,71124.0,1241341.0,5729.610155,102969,82
81,83 years,73524.0,1151190.0,6386.782373,102969,83


In [13]:
fig = go.Figure()
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Population"]))
fig.add_trace(go.Bar(x=usa_total["age"], y=usa_total["Rate"]))

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=usa_total["age"], y=usa_total["Rate"], mode="lines"))

In [15]:
df["Ten-Year Age Groups Code"].unique()

array(['85+', '75-84', '65-74', '55-64', '1', '45-54', '35-44', '25-34',
       '15-24', '1-4', '5-14'], dtype=object)

In [16]:
child = state_deaths[state_deaths["Ten-Year Age Groups Code"] == '5-14']
part = Partisian.get_state_part_score()
child = pd.merge(child, part, left_on="State_Abv", right_on="State")


In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=child['Part_Score'], y=child['Rate'],mode='markers',name='Child'))

In [18]:
fig = go.Figure()
fig.add_trace(go.Bar(x=child["State_Abv"], y=child["Rate"], marker=dict(color=child["Part_Score"], colorscale="Bluered" ), name="Child"))

In [19]:
child

,State_x,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips_x,Year_x,State_Abv,State_y,fips_y,Part_Score,Year_y,category
0,Mississippi,5-14 years,5-14,98.0,395659.0,24.8,24.77,28,2019,MS,MS,28,0.800000,2024,Right
1,Idaho,5-14 years,5-14,59.0,254788.0,23.2,23.16,16,2019,ID,ID,16,1.000000,2024,Right
2,Alaska,5-14 years,5-14,22.0,100662.0,21.9,21.86,02,2019,AK,AK,02,0.466667,2024,Right
3,Alabama,5-14 years,5-14,126.0,608466.0,20.7,20.71,01,2019,AL,AL,01,0.866667,2024,Right
4,Wyoming,5-14 years,5-14,15.0,76734.0,Unreliable,19.55,56,2019,WY,WY,56,0.866667,2024,Right
5,Oklahoma,5-14 years,5-14,101.0,538048.0,18.8,18.77,40,2019,OK,OK,40,0.866667,2024,Right
6,Louisiana,5-14 years,5-14,113.0,607936.0,18.6,18.59,22,2019,LA,LA,22,0.333333,2024,Right
7,Montana,5-14 years,5-14,24.0,129305.0,18.6,18.56,30,2019,MT,MT,30,0.200000,2024,Center
8,Arkansas,5-14 years,5-14,70.0,393213.0,17.8,17.80,05,2019,AR,AR,05,0.333333,2024,Right
9,Kentucky,5-14 years,5-14,97.0,560417.0,17.3,17.31,21,2019,KY,KY,21,0.200000,2024,Center


In [20]:
import GraphGenerator
def get_age_death(df, age):
    age_df = df[df["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    X = age_df['Part_Score'].values.reshape(-1,1)
    y = age_df['Rate']
    
    reg = LinearRegression().fit(X, y, sample_weight=age_df['Population'])
    x_range = np.linspace(X.min(), X.max())
    y = reg.predict(x_range.reshape(-1,1))
    
    fig = make_subplots(rows=1, cols=2, vertical_spacing= 0.01, horizontal_spacing= 0.01,  subplot_titles=("",""), column_widths=[0.8, 0.2], shared_yaxes=True)
    
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers', marker=dict(color=age_df["Part_Score"], colorscale="Bluered", size=age_df["Population"]/(age_df["Population"].max()/50)), name='Child', hovertext=age_df["State_x"]), row=1, col=1)
    
    hist_x = []
    for index, row in age_df.iterrows():
        hist_x.extend([row["Rate"] for i in range(int(round(row["Population"]/1000)))])
        row["Rate"]
    #fig.add_trace(go.Violin(y=hist_x, side="positive", name="Death Rate Distribution"), row=1, col=2)
    fig.add_trace(go.Violin(y=age_df['Rate'], side="positive",  name="Death Rate Distribution"), row=1, col=2)
    
    
    fig.add_trace(go.Scatter(x=x_range, y=y, mode='lines',  line=dict(color="green", dash="dash"), name='Best Fit Line'), row=1, col=1)
    
    fig.update_layout(title="Partisanship Score vs. Death Rate for " + age + " Year Olds", xaxis1_title="Partisanship Score", yaxis1_title="Death Rate (per 100,000)")
    fig.update_yaxes(showticklabels=False, row=1, col=2)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
    fig.update_layout(height=300,width=1200, plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0, r=0, t=25, b=0))
    
    fig.show()


In [21]:
get_age_death(state_deaths,"85+")

In [22]:
for age in state_deaths["Ten-Year Age Groups Code"].unique():
    None
    #get_age_death(state_deaths,age)

In [23]:

fig = go.Figure()

for age in state_deaths["Ten-Year Age Groups Code"].unique():
    age_df = state_deaths[state_deaths["Ten-Year Age Groups Code"] == age]
    part = Partisian.get_state_part_score()
    age_df = pd.merge(age_df, part, left_on="State_Abv", right_on="State")
    fig.add_trace(go.Scatter(x=age_df['Part_Score'], y=age_df['Rate'],mode='markers',name=age))
    #fig.add_trace(go.Bar(x=age_df["State_Abv"], y=age_df["Rate"], marker=dict(color=age_df["Part_Score"], colorscale="Bluered" ), name=age))

fig.show()

In [24]:
teen = df[df["Ten-Year Age Groups Code"] == "85+"]
teen.sample(3)
teen_all = teen#[teen["ICD-10 Original"].str.contains("#") == True]
ny = teen_all[teen_all["State"] == "New York"]
ny.dropna(inplace=True, subset=["ICD-10 113 Cause List"])
ny.tail(3)

,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv
14967,New York,"Bronchitis, chronic and unspecified (J40-J42)",GR113-083,85+ years,85+,14.0,460001.0,Unreliable,3.04,36,2019,NY
15153,New York,"Accidental exposure to smoke, fire and flames ...",GR113-121,85+ years,85+,13.0,460001.0,Unreliable,2.83,36,2019,NY
14875,New York,Acute and subacute endocarditis (I33),GR113-065,85+ years,85+,10.0,460001.0,Unreliable,2.17,36,2019,NY


In [25]:
f = open("icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


FileNotFoundError: [Errno 2] No such file or directory: 'icd-10-Structure.json'

In [ ]:
#ny.loc[15081]

In [ ]:
ny["ICD-10 113 Cause List"] = pd.Categorical(ny["ICD-10 113 Cause List"], categories = list(heirarchy.keys()))
ny = ny.sort_values(by=["ICD-10 113 Cause List"])
ny["ICD-10 113 Cause List"].unique()
ny.dropna(subset=["ICD-10 113 Cause List"], inplace=True)

In [26]:
ny

,State,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,fips,Year,State_Abv
14797,New York,Major cardiovascular diseases (I00-I78),GR113-053,85+ years,85+,24287.0,460001.0,5279.8,5279.77,36,2019,NY
14807,New York,"#Diseases of heart (I00-I09,I11,I13,I20-I51)",GR113-054,85+ years,85+,19770.0,460001.0,4297.8,4297.82,36,2019,NY
14830,New York,Ischemic heart diseases (I20-I25),GR113-058,85+ years,85+,13239.0,460001.0,2878.0,2878.04,36,2019,NY
14848,New York,Other forms of chronic ischemic heart disease ...,GR113-061,85+ years,85+,10924.0,460001.0,2374.8,2374.78,36,2019,NY
14861,New York,All other forms of chronic ischemic heart dise...,GR113-063,85+ years,85+,9712.0,460001.0,2111.3,2111.30,36,2019,NY
...,...,...,...,...,...,...,...,...,...,...,...,...
15037,New York,Acute and rapidly progressive nephritic and ne...,GR113-098,85+ years,85+,15.0,460001.0,Unreliable,3.26,36,2019,NY
15020,New York,Alcoholic liver disease (K70),GR113-094,85+ years,85+,15.0,460001.0,Unreliable,3.26,36,2019,NY
14967,New York,"Bronchitis, chronic and unspecified (J40-J42)",GR113-083,85+ years,85+,14.0,460001.0,Unreliable,3.04,36,2019,NY
15153,New York,"Accidental exposure to smoke, fire and flames ...",GR113-121,85+ years,85+,13.0,460001.0,Unreliable,2.83,36,2019,NY


In [27]:
labels = ["All"]
labels.extend(list(ny["ICD-10 113 Cause List"]))

values = [0]
values.extend(list(ny.Rate))


parents = []
for i,label in enumerate(labels):
    if label == "All":
        parents.append("")
    else:
        try:
            parents = flat_heirarchy[label]["parents"]
            if len(parent) > 0:
                parents.append(parent[0])
            else:
                parents.append("")
                #values[i] = values[i]-values[parents.index(heirarchy[label])]
            
        except:
            parents.append("")  



In [28]:
test_labels = labels[:11]
test_parents = parents[:11]
test_values = values[:11]
test_values[0] = sum(test_values[1:])

In [29]:
fig =go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
))

fig.show()

In [71]:
df = pd.read_csv(r"../data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"])

In [32]:
df["Ten-Year Age Groups Code"].unique()

array(['85+', '75-84', '65-74', '55-64', '1', '45-54', '35-44', '25-34',
       '15-24', '1-4', '5-14'], dtype=object)

In [70]:
f = open("../icd-10-Structure.json")
heirarchy = json.load(f)
heirarchy

f = open("../icd-10-flat-Structure.json")
flat_heirarchy = json.load(f)


In [91]:
def load_state_cod(state="Michgian"):
    f = open("../icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    df_heirarchy = pd.DataFrame(flat_heirarchy)
    #rotate Matrix
    df_heirarchy = df_heirarchy.transpose()
    df_heirarchy.reset_index(inplace=True)
    df_heirarchy.rename(columns={"index":"ICD-10 113 Cause List"}, inplace=True)
    df_heirarchy

    df = pd.read_csv(r"../data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    
    #state_df = state_df[state_df["Ten-Year Age Groups Code"] ==age]
    df["Rate"] = (df["Deaths"] / df["Population"])*100000
    df["Rate"] = np.round(df["Rate"], 2)
    df["Child_Adj_Rate"] = df["Rate"]
    if(state != "All"):
        df= df[df["State"] == state]


    df_final = pd.merge(df, df_heirarchy, on="ICD-10 113 Cause List")
    df_final = df_final.drop(columns=["ICD-10 113 Cause List Code", "ranges", "chapters", "base", "Notes"])
    return df_final

In [116]:
def cod_v_age_box(cod, state):
    df = load_state_cod("All")
    df = df[df["ICD-10 113 Cause List"] == cod]
    df = df.sort_values(by=["Ten-Year Age Groups Code"])
    # Create box plot  
    fig = px.box(df, x="Ten-Year Age Groups Code", y="Rate", color="Ten-Year Age Groups Code", hover_data=["State", "ICD-10 113 Cause List"])
    fig.show()
    

In [117]:
cod_v_age_box("Renal failure (N17-N19)", "Michigan")

In [154]:
df = load_state_cod("All")
df = df[df["ICD-10 113 Cause List"] == "#Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)"]
df["Start Age"] = df["Ten-Year Age Groups Code"].str.split("-").str[0].str.replace("+","")
df["Start Age"] = df["Start Age"].astype(int)

df = df.sort_values(by=["Start Age"])

fig = px.box(df, x="Ten-Year Age Groups Code", y="Rate", color="Ten-Year Age Groups Code", hover_data=["State", "ICD-10 113 Cause List"])

df = df[df["State"] == "Alabama"]
fig.add_trace(go.Scatter(
    x = df["Ten-Year Age Groups Code"],
    y=df["Rate"],
    mode = "markers+lines",
    marker=dict(color = "red"),
    name = "Alabama"
    
))
fig.show()

C:\Users\zeusg\AppData\Local\Temp/ipykernel_38384/1163915242.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [155]:
df.sort_values(by="Start Age")

,State,State Code,ICD-10 113 Cause List,Ten-Year Age Groups,Ten-Year Age Groups Code,Deaths,Population,Crude Rate,Rate,Child_Adj_Rate,common_name,children,parents,Start Age
20891,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",5-14 years,5-14,10.0,608466.0,Unreliable,1.64,1.64,Suicide,[Intentional self-harm (suicide) by other and ...,[],5
20892,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",15-24 years,15-24,94.0,635707.0,14.8,14.79,14.79,Suicide,[Intentional self-harm (suicide) by other and ...,[],15
20893,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",25-34 years,25-34,129.0,647229.0,19.9,19.93,19.93,Suicide,[Intentional self-harm (suicide) by other and ...,[],25
20894,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",35-44 years,35-44,157.0,593306.0,26.5,26.46,26.46,Suicide,[Intentional self-harm (suicide) by other and ...,[],35
20895,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",45-54 years,45-54,137.0,616616.0,22.2,22.22,22.22,Suicide,[Intentional self-harm (suicide) by other and ...,[],45
20896,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",55-64 years,55-64,119.0,657667.0,18.1,18.09,18.09,Suicide,[Intentional self-harm (suicide) by other and ...,[],55
20897,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",65-74 years,65-74,84.0,501447.0,16.8,16.75,16.75,Suicide,[Intentional self-harm (suicide) by other and ...,[],65
20898,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",75-84 years,75-84,51.0,256847.0,19.9,19.86,19.86,Suicide,[Intentional self-harm (suicide) by other and ...,[],75
20899,Alabama,1.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",85+ years,85+,23.0,91543.0,25.1,25.12,25.12,Suicide,[Intentional self-harm (suicide) by other and ...,[],85


In [72]:
def add_children(name, child, order):
    f = open("../icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    order.append(name)
    if len(child["children"]) > 0: 
        for grand_child in child["children"]:
            add_children(grand_child, flat_heirarchy[grand_child], order)
def get_depth_order(flat_heirarchy):
    order = []
    for key, value in flat_heirarchy.items():
        if len(value["parents"]) == 0:
            order.append(key)
            for children in value["children"]:
                add_children(children, flat_heirarchy[children],order)
    return order

In [73]:
def get_age_cod(df, age, all_ages=False):
    f = open("../icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)
    order = get_depth_order(flat_heirarchy)
    df = df[df["Ten-Year Age Groups Code"] ==age]
    for e in range(5):
        for i, row in df.iterrows():
            children_total = 0
            children_values =[]
            if len(row["children"]) > 0:
                for child in row["children"]:
                    try:
                        children_total += df[df["ICD-10 113 Cause List"] == child]["Child_Adj_Rate"].values[0]
                        children_values.append(df[df["ICD-10 113 Cause List"] == child]["Child_Adj_Rate"].values[0])
                    except:
                        None
            df.at[i, "Child_Adj_Rate"] = max(row["Child_Adj_Rate"], children_total)
    df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].astype("category")
    df["ICD-10 113 Cause List"] = df["ICD-10 113 Cause List"].cat.set_categories(order, ordered=True)
    df = df.sort_values("ICD-10 113 Cause List")
    
    labels = [age]
    parents = [""]
    if all_ages:
        parents = ["All Ages"]
    values = [0]
    colors = ["white"]
    for i, row in df.iterrows():
        labels.append(row["ICD-10 113 Cause List"].split("(")[0] + ": " + str(age))
        if len(row["parents"]) > 0:
            parents.append(row["parents"][0].split("(")[0] + ": " + str(age))
        else: 
            parents.append(age)
            values[0] += row["Child_Adj_Rate"]
        values.append(row["Child_Adj_Rate"])
    return labels, parents, values, df, values[0]
    
df_final = load_state_cod()
labels, parents, values, df, total = get_age_cod(df_final, age, all_ages=True)

In [74]:
def get_all_age_cod(df):
    all_labels = ["All Ages"]
    all_parents = [""]
    all_values = [0]
    for age in df["Ten-Year Age Groups Code"].unique():
        labels, parents, values, df_age, total = get_age_cod(df, age, True)
        all_labels.extend(labels)
        all_parents.extend(parents)
        all_values.extend(values)
        all_values[0] += total
    return all_labels, all_parents, all_values
    
final_labels, final_parents, final_values = get_all_age_cod(df_final)    

In [75]:
fig = go.Figure(go.Treemap(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
    textinfo = "label+value+percent root",
    textposition = 'middle center',
    textfont = dict(family="Arial", size=16, color = '#FFFFFF')
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
    
)

fig.show()
fig.write_html("CauseOfDeathTest.html")

In [76]:
fig = go.Figure(go.Sunburst(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()

In [77]:
fig = go.Figure(go.Icicle(
    branchvalues = "total",
    labels = final_labels,
    parents =  final_parents,
    values=final_values,
    tiling = dict(
            orientation='v',
        ),
    textposition = 'middle center',
    textfont = dict(family="Arial", size=18, color = '#FFFFFF')
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))


fig.update_layout(
    #uniformtext=dict(minsize=8, mode='hide'),
    autosize=False,
    width=2100,
    height=1050,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
    ),
)

fig.show()
fig.write_html("CauseOfDeathIcicle.html")

#here

In [78]:
def load_state_cod_old(state, age_group = '75-84'):
    df = pd.read_csv(r"data/Life/Deaths/StateDeathsAge.txt", delimiter="	", na_values = ['Not Applicable'])
    df = df.dropna(subset=["State","ICD-10 113 Cause List Code", "Population"]) 
    state_df= df[df["State"] == state]
    state_df = state_df[state_df["Ten-Year Age Groups Code"] =='75-84']
    state_df["Rate"] = (state_df["Deaths"] / state_df["Population"])*100000
    state_df["Rate"] = np.round(state_df["Rate"], 2)
    state_df["Child_Adj_Rate"] = state_df["Rate"]
    
    state_df["Parent"] = ""
    state_df["Children"] = ""
    state_df["Common Name"] = ""
    
    
    f = open("icd-10-flat-Structure.json")
    flat_heirarchy = json.load(f)

    targets =[]
    sources = []
    values = []

    sources.append("")
    targets.append("All")
    values.append(0)
    values.extend(list(state_df.Rate))
    


    for i, row in state_df.iterrows():
        try:
            structure = flat_heirarchy[row["ICD-10 113 Cause List"]]
            state_df.at[i, "Common Name"] = structure["common_name"]
            targets.append(structure["common_name"])
            structure["Rate"] = row["Rate"]
            
            if len(structure["parents"])  == 1:
                parent = flat_heirarchy[structure["parents"][0]]["common_name"]
                state_df.loc[i, "Parent"] = parent
                sources.append(parent)
            else:
                state_df.loc[i, "Parent"] = "All"
                sources.append("All")
        except Exception as e:
            print(e)
        if len(structure["children"]) > 0:
            state_df.at[i, "Children"] = structure["children"]
             
    for i, source in enumerate(sources):
        if source == "":
            None
        else:
            try:
                parent_index = targets.index(source)
                values[parent_index] = max( values[parent_index] - values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[parent_index]
                
            except:
                values[i] = max(values[i],0)
                state_df.loc[state_df['Common Name'] == targets[parent_index], 'Child_Adj_Rate'] = values[i]
                
    values[0] = 5000
    size = 50
    plot_dict = {"values":values[:size], "targets":targets[:size], "parents":sources[:size]}
    return state_df, plot_dict

In [55]:
df_final = load_state_cod(state="All")
state_pop = pd.read_csv(r"../data/Life/Deaths/State_Pop.txt", delimiter="	", na_values = ['Not Applicable'])
state_pop = clean_state_data(state_pop)[["State","Population","Deaths","State_Abv"]]
state_pop.rename(columns={"Deaths":"Total Deaths"}, inplace=True)

In [56]:
cardio = df_final[df_final["ICD-10 113 Cause List"] == "Major cardiovascular diseases (I00-I78)"].groupby("State").agg({"Deaths":"sum"}).reset_index()
cardio = pd.merge(cardio, state_pop, left_on="State", right_on="State")
cardio["Rate"] = cardio["Deaths"]/cardio["Population"]*100000
cardio = cardio.sort_values("Rate", ascending=False)
cardio

,State,Deaths,Population,Total Deaths,State_Abv,Rate
48,West Virginia,6670.0,1792147.0,23404.0,WV,372.179291
3,Arkansas,10813.0,3017804.0,32888.0,AR,358.306901
0,Alabama,17559.0,4903185.0,54108.0,AL,358.114165
24,Mississippi,10511.0,2976149.0,32964.0,MS,353.174522
36,Oklahoma,13432.0,3956971.0,40930.0,OK,339.451565
22,Michigan,32783.0,9986857.0,99084.0,MI,328.261434
35,Ohio,38109.0,11689100.0,123717.0,OH,326.021678
38,Pennsylvania,41270.0,12801989.0,133983.0,PA,322.371781
42,Tennessee,21570.0,6829174.0,71935.0,TN,315.850790
25,Missouri,19026.0,6137428.0,62401.0,MO,309.999563


In [80]:
def nation_single_cod(cod = "Major cardiovascular diseases (I00-I78)", state = "Michigan"):
    # Load Data
    df = pd.read_csv(r"..\data\Life\Deaths\2019CuaseOfDeath.txt", delimiter="	", na_values = ['Not Applicable'])
    df = clean_state_data(df)
    # Filter by COD
    cod = cod.split(":")[0].strip()
    df = df[df["ICD-10 113 Cause List"].str.contains(cod)].sort_values("Rate", ascending=False)
    
    
    fig = go.Figure()
    fig.add_trace(go.Box(
        x=df["Rate"], 
        name=cod.split("(")[0],
        boxpoints='all', # can also be outliers, or suspectedoutliers, or False
        jitter=0.3, # add some jitter for a better separation between points
        pointpos=-1.8, # relative position of points wrt box
        hovertext=df["State_Abv"]

    ))
    fig.add_vline(x=df[df["State"]==state]["Rate"].values[0], line_width=3, line_dash="dash", line_color="green")
    fig.show()
    return df


In [81]:
cancer = nation_single_cod(cod = "#Malignant neoplasms : 85+", state="Texas")

In [87]:
fig = go.Figure()
fig.add_trace(go.Box(
    x=cancer["Rate"], 
    name="Cancer",
    boxpoints='all', # can also be outliers, or suspectedoutliers, or False
    jitter=0.3, # add some jitter for a better separation between points
    pointpos=-1.8, # relative position of points wrt box
    hovertext=cancer["State_Abv"]
            
))
fig.add_vline(x=226.99, line_width=3, line_dash="dash", line_color="green")


fig.show()

In [88]:
fig = go.Figure()
fig.add_trace(go.Violin(
    x=cancer["Rate"], 
    name="Cancer",
    points='all', # can also be outliers, or suspectedoutliers, or False
    jitter=0.3, # add some jitter for a better separation between points
    pointpos=-1.8, # relative position of points wrt box
    hovertext=cancer["State_Abv"],
    side="positive"
            
))
fig.add_vline(x=226.99, line_width=3, line_dash="dash", line_color="green")


fig.show()

In [89]:
cancer

,State,Year,Year Code,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Rate,fips,State_Abv
5013,West Virginia,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,4604.0,1792147.0,256.9,249.5,264.3,256.90,54,WV
1983,Maine,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,3413.0,1344212.0,253.9,245.4,262.4,253.90,23,ME
1769,Kentucky,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,9975.0,4467673.0,223.3,218.9,227.7,223.27,21,KY
2512,Mississippi,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,6587.0,2976149.0,221.3,216.0,226.7,221.33,28,MS
4702,Vermont,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,1378.0,623989.0,220.8,209.2,232.5,220.84,50,VT
3970,Pennsylvania,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,27746.0,12801989.0,216.7,214.2,219.3,216.73,42,PA
3643,Ohio,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,25170.0,11689100.0,215.3,212.7,218.0,215.33,39,OH
309,Arkansas,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,6482.0,3017804.0,214.8,209.6,220.0,214.79,05,AR
746,Delaware,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,2074.0,973764.0,213.0,203.8,222.2,212.99,10,DE
916,Florida,2019,2019.0,#Malignant neoplasms (C00-C97),GR113-019,45583.0,21477737.0,212.2,210.3,214.2,212.23,12,FL
